## TFX Pipelines

#### Generate Examples

In [7]:
# Temporary commands to unzip the zip file in Google Cloud Storage
# ! gsutil -m cp gs://text-analysis-323506/train_data/train_val.zip ./
# ! unzip train_val.zip
# ! gunzip *.csv.gz
# ! gsutil -m mv *.csv gs://text-analysis-323506/train_data/

In [8]:
# ! pip3 install tfx

In [57]:
import time
import os

import tensorflow as tf

from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2
from tfx.components import StatisticsGen

In [49]:
tf.__version__

'2.6.2'

In [24]:
tfx.__version__

'1.4.0'

In [34]:
DATA_ROOT = 'gs://text-analysis-323506/train_data/'

In [35]:
PIPELINE_NAME = 'sentiment-analysis'
PIPELINE_ROOT = os.path.join('/home/jupyter/', PIPELINE_NAME, time.strftime("%Y%m%d_%H%M%S"))
os.makedirs(PIPELINE_ROOT, exist_ok=True)

In [36]:
context = InteractiveContext(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=None)

### CSV Example Generator

In [37]:
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[        
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=4),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))

In [38]:
example_gen = CsvExampleGen(
    input_base=DATA_ROOT,
    output_config=output_config)

In [ ]:
context.run(example_gen)

In [42]:
examples_uri = example_gen.outputs['examples'].get()[0].uri

In [45]:
tfrecord_filenames = [os.path.join(examples_uri, 'Split-train', name)
                      for name in os.listdir(os.path.join(examples_uri, 'Split-train'))]

In [50]:
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

2021-11-21 18:14:07.697951: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [54]:
for tfrecord in dataset.take(2):
    example = tf.train.Example()
    example.ParseFromString(tfrecord.numpy())
    for name, feature in example.features.feature.items():
        if feature.HasField('bytes_list'):
            value = feature.bytes_list.value
        if feature.HasField('float_list'):
            value = feature.float_list.value
        if feature.HasField('int64_list'):
            value = feature.int64_list.value
        print('{}: {}'.format(name, value))
    print('******')

labels: [1]
input: [b'myattorney home business looking account prose litigant state federal lawsuite maintain hisher files']
******
labels: [0]
input: [b'great book liked book sarah plain tall authors name patricia maclachen characters names caleb sarah anna papa though book interestingit first started caleb siting fire asking questions mama singing songs anna explaining born mama died sarah answered papas letter came live sarah taught caleb swim anybody gets book really enjoy']
******


#### Train and eval datasets have been created properly !

### Statistics Generator

In [58]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

In [ ]:
context.run(statistics_gen)

In [ ]:
context.show(statistics_gen.outputs['statistics'])